**Fake Job Prediction**

Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras


# Web Scrapping

In [ ]:
import requests
import csv
import time
from bs4 import BeautifulSoup
import pandas as pd


# Define the locations and job search keywords
#locations = [
    # Asia
    #Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Ahmedabad', 'Chennai', 'Kolkata','Hong Kong',
    # Africa
     #'Johannesburg', 'Cape Town', 'Nairobi','Accra','Kampala','Addis Ababa', 'Durban', 'Pretoria', 'Kimberley'
#]
#job_search_keywords = [
    # Tech industry
    #'Data+Scientist', 'Software+Engineer', 'Full+Stack+Developer', 'Front+End+Developer', 'Back+End+Developer',
    #'DevOps+Engineer', 'Database+Administrator', 'Data+Analyst',
    # Finance industry
    #'Financial+Analyst', 'Accountant', 'Financial+Advisor', 'Auditor', 'Insurance+Agent',
    #'Investment+Banker', 'Tax+Consultant', 'Budget+Analyst', 'Cost+Estimator', 'Loan+Officer',
    # Healthcare industry
    #'Nurse', 'Physician', 'Pharmacist', 'Dietitian', 'Physical+Therapist',
    #'Radiologic+Technologist', 'Medical+Technologist', 'Home+Health+Aide', 'Medical+Transcriptionist', 'Phlebotomist',
    # Education industry
    #'Teacher', 'Teaching+Assistant',
    #'Childcare+Worker',  'Preschool+Teacher', 'College+Counselor', 'School+Psychologist',
    # Marketing industry
    #'Marketing+Manager', 'SEO+Specialist', 'Content+Marketing+Manager', 'Brand+Manager', 'Public+Relations+Specialist',
    #'Media+Planner', 'Product+Manager', 'Digital+Marketing+Manager', 'Social+Media+Manager', 'Marketing+Analyst'
#]
locations = ['USA']
job_search_keywords = ['Business+Analyst']
# Loop through each location and job search keyword and extract the job data
job_data = []
for location in locations:
    for job_search_keyword in job_search_keywords:
        # Define the URL of the job search page
        url = f'https://www.linkedin.com/jobs/search/?keywords={job_search_keyword}&location={location}'

        # Send a GET request to the URL and store the response
        response = requests.get(url)

        # Parse the HTML content of the response using Beautiful Soup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the job listings on the page
        #job_listings = soup.find_all('li', class_='job-search-card')

        # print(soup)
        # Find all of the job listings on the page
        job_listings = soup.find_all(class_='job-search-card')
        # For each job listing, extract the title, company, and location
        for i, listing in enumerate(job_listings):
            title = listing.find('h3').text.strip()
            company = listing.find('h4').text.strip()
            location = listing.find('span', class_='job-search-card__location').text.strip()
            job_link = listing.find("a", href=True)["href"]
            resp = requests.get(job_link)
            soup=BeautifulSoup(resp.text,'html.parser')
            #time.sleep(5)  # Pause to avoid overwhelming the server
            job_criteria_list = soup.find("ul", {"class": "description__job-criteria-list"})
            if job_criteria_list:
               job_criteria = job_criteria_list.find_all("li")
            else:
               break
            try:
                level=job_criteria[0].text.replace("Seniority level","").strip()
            except:
                level=None
            try:
                empType=job_criteria[1].text.replace("Employment type","").strip()
            except:
                empType=None
            try:
                empFunction=job_criteria[2].text.replace("Job function","").strip()
            except:
                empFunction=None
            try:
                industries=job_criteria[3].text.replace("Industries","").strip()
            except:
                industries=None
            try:
                salary=soup.find("div", {"class": "salary compensation__salary"}).text.strip()
            except:
                salary=None

            job_data.append({'job_id':i, 'title': title, 'location': location , 'department':None, 'salary_range': salary ,
                             'company_profile':company,'description':None, 'requirements':None, 'benefits':None,
                             'telecommuting':0 ,'has_company_logo':0 , 'has_questions':0 , 'employment_type': empType,
                             'required_experience': level,'required_education':None,'industry': industries,
                             'function': empFunction , 'fraudulent':0 })
            # Define the file name and header row

# Convert the job data to a pandas DataFrame
df_linkedin = pd.DataFrame(job_data)

# Save the DataFrame to a CSV file
df_linkedin.to_csv('jobs_linkedin.csv', index=False, encoding='utf-8')

print("Web Scrapping done!")


Web Scrapping done!


# Importing DataSet

In [ ]:
# 1st dataset ( main feed )
jobs_file = tf.keras.utils.get_file(
    fname="./fraudulent_jobs.csv",
    origin="https://uofi.box.com/shared/static/sfw0eqvj7q49vmexpztqke7xzhspevnb.csv"
)

df=pd.read_csv(jobs_file)

df.head()

50061601/50061601 [==============================] - 3s 0us/step


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


Training Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


df['full_text'] = df['title'] + " " + df['location'] + " " + df['department'] + " " + df['company_profile'] + " " + df['description'] + " " + df['requirements'] + " " + df['benefits']
df['full_text'] = df['full_text'].fillna('')
# Use TF-IDF to convert the text data into numerical data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['full_text'])

# The 'fraudulent' column is the target variable
y = df['fraudulent']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier on the training data
# Train a Random Forest Classifier on the training data with adjusted parameters
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)


# Use the model to predict the 'fraudulent' column for the test data
y_pred = model.predict(X_test)

# Print a classification report to see how well the model performed
print(classification_report(y_test, y_pred))

# Load the other dataset
jobs_linkedin = pd.read_csv('/content/jobs_linkedin.csv')

# Combine all the text data into a single column
#combine all text into 1 column - full_text
jobs_linkedin['full_text'] = jobs_linkedin['title'] + " " + jobs_linkedin['location'] + " " + jobs_linkedin['department'] + " " + jobs_linkedin['company_profile'] + " " + jobs_linkedin['description'] + " " + jobs_linkedin['requirements'] + " " + jobs_linkedin['benefits']
jobs_linkedin['full_text'] = jobs_linkedin['full_text'].fillna('')

# Use the same TF-IDF vectorizer to transform the text data into numerical data
X_jobs_linkedin = vectorizer.transform(jobs_linkedin['full_text'])

# Use the trained model to predict the 'fraudulent' column for the other dataset
jobs_linkedin['fraudulent'] = model.predict(X_jobs_linkedin)

# Save the DataFrame to a CSV file
jobs_linkedin.to_csv('jobs_linkedin_with_fraudulent.csv', index=False, encoding='utf-8')

print("Data generation done!")

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      3395
           1       1.00      0.12      0.21       181

    accuracy                           0.96      3576
   macro avg       0.98      0.56      0.59      3576
weighted avg       0.96      0.96      0.94      3576

Data generation done!


Upload Local File(s)

In [10]:
from IPython.display import display, FileLink
import ipywidgets as widgets
from IPython.display import display

# Create a file upload widget
upload_widget = widgets.FileUpload()

# Display the upload widget
display(upload_widget)

# Define a function to handle the uploaded file
def handle_upload(change):
    uploaded_filename = next(iter(upload_widget.value))
    content = upload_widget.value[uploaded_filename]['content']

    # Write the content to a local file
    local_filename = f'./{uploaded_filename}'
    with open(local_filename, 'wb') as f:
        f.write(content)

    print(f'File {uploaded_filename} uploaded and saved as {local_filename}')

# Attach the function to the widget's event
upload_widget.observe(handle_upload, names='_counter')


FileUpload(value={}, description='Upload')

In [ ]:
# Import File from local memory after upload.
df2=pd.read_csv('/content/CompiledjobListNigeria.csv')

# Find the maximum value in the 'Code' column of df
max_code = df['job_id'].max()

# Check if 'job_id' column exists in df, and add it with index row if not
if 'job_id' not in df2.columns:
    df2['job_id'] = df2.reset_index().index

# Check if 'ColumnName' column exists in df, and add it with index row if not
if 'benefits' not in df2.columns:
    df2['benefits'] = None

# Check if 'ColumnName' column exists in df, and add it with index row if not
if 'function' not in df2.columns:
    df2['function'] = None

# Check if 'ColumnName' column exists in df, and add it with index row if not
if 'industry' not in df2.columns:
    df2['industry'] = None

# Check if 'ColumnName' column exists in df, and add it with index row if not
if 'required_education' not in df2.columns:
    df2['required_education'] = None

# Check if 'ColumnName' column exists in df, and add it with index row if not
if 'required_experience' not in df2.columns:
    df2['required_experience'] = None

# Increment the 'job_id' column in df2 by the maximum value from df + 1
df2['job_id'] = df2['job_id'].apply(lambda x: x + max_code + 1)

# Add a new column 'has_company_logo' with random values of 0.0 or 1.0
df2['has_company_logo'] = np.random.choice([0, 1], size=len(df2))

# Add a new column 'has_questions' with random values of 0.0 or 1.0
df2['has_questions'] = np.random.choice([0, 1], size=len(df2))

# Add a new column 'telecommuting' with random values of 0.0 or 1.0
df2['telecommuting'] = np.random.choice([0, 1], size=len(df2))

# Rename the columns in the second dataframe to match the columns in the first dataframe
df2.rename(columns={'job_title': 'title',
                    'company_name': 'company_profile',
                    'company_desc': 'description',
                    'job_requirement': 'requirements',
                    'salary': 'salary_range',
                    'label': 'fraudulent'}, inplace=True)

# Remove the 'job_desc' column


df2 = df2.drop('job_desc', axis=1)

column_order = ['job_id', 'title',	'location',	'department',	'salary_range',	'company_profile',	'description',	'requirements',	'benefits',	'telecommuting',	'has_company_logo',	'has_questions',	'employment_type',	'required_experience',	'required_education',	'industry',	'function',	'fraudulent']
df2 = df2[column_order]

df2.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/CompiledjobListNigeria.csv'

In [ ]:
# Merge datas
merged_df = pd.merge(df, df2, on=['job_id', 'title',	'location',	'department',	'salary_range',	'company_profile',	'description',	'requirements',	'benefits',	'telecommuting',	'has_company_logo',	'has_questions',	'employment_type',	'required_experience',	'required_education',	'industry',	'function',	'fraudulent'], how='outer')
print(merged_df)

In [ ]:
# Import Scrapping File from local memory after scrapping process.
df3=pd.read_csv('/content/jobs_linkedin.csv')

# Find the maximum value in the 'Code' column of df
max_code = merged_df['job_id'].max()

# Increment the 'job_id' column in df2 by the maximum value from df + 1
df3['job_id'] = df3['job_id'].apply(lambda x: x + max_code )

# Convert multiple columns to object type
columns_to_convert = ['department', 'salary_range', 'description', 'requirements', 'benefits', 'required_education']
df3[columns_to_convert] = df3[columns_to_convert].astype(str)

# Remove the 'Link' column
#df2 = df2.drop('Link')

column_order = ['job_id', 'title',	'location',	'department',	'salary_range',	'company_profile',	'description',	'requirements',	'benefits',	'telecommuting',	'has_company_logo',	'has_questions',	'employment_type',	'required_experience',	'required_education',	'industry',	'function',	'fraudulent']
df3 = df3[column_order]

# Merge datas
final_merged_df = pd.merge(merged_df, df3, on=['job_id', 'title',	'location',	'department',	'salary_range',	'company_profile',	'description',	'requirements',	'benefits',	'telecommuting',	'has_company_logo',	'has_questions',	'employment_type',	'required_experience',	'required_education',	'industry',	'function',	'fraudulent'], how='outer')
print(final_merged_df)

Il y a au total 18 variables, et elles peuvent être réparties en quatre catégories : Texte, Catégorique, Numérique et Autre. les caractéristiques qui correspondent à chaque catégorie:

**Text** : Title, Location, Department, Company Profile, Description, Requirements, Benefits.

**Categorical** :  Employment Type, Required Experience, Required Education, Industry, Function.

**Numeric** :  Job Id, Has Company Logo, Telecommuting, Has Questions, Fraudulent.

**Other** :  Salary Range.


                         *Text Variables*

**Title**: title of the job ad entry (11,231).

 **Location**: geographical location of the job ad (3105).

**Department**: corporate department (1337).

**Company Profile**: brief company description (1709).

**Description**: details description of the job ad (14,801).

**Requirements**: enlisted requirements for the job opening (11,968).

**Benefits**: enlisted benefits offered by the employer (6205).

**Salary Range**: indicative salary range (874).

*Note: this value is a string in the original dataset but will be split into two numeric values during pre-processing

              *Categorical Variables*

**Employment Type**: full-time, part-time, etc.

**Required Experience**: entry-level, etc.

**Required Educatio**n: Bachelor, etc.

**Industry**: automotive, IT, etc.

 **Function**: consulting, research, etc.


                            *Numeric Variables*

**Job Id**: identification number for the job posting (18,091).

 **Telecommuting**: indicates whether posting mentions telecommuting.

**Has Company Logo**: indicates whether company logo is present.

**Has Questions**: indicates whether screening questions are present.

**Fraudulent**: indicates whether posting is fake.

# Exploring the DataSet

In [ ]:
final_merged_df.info()

* Total Entries: The dataset has a total of 17,880 entries.

* The columns of the dataset

* Missing Values: Some columns have missing values, such as 'location,' 'department,' 'salary_range,' 'company_profile,' and others. The number of non-null entries in each column is provided.

* Data Types: The data types include integers for numerical columns and objects for textual columns.

In [ ]:
# get unique values from the dataset
final_merged_df.nunique()

In [ ]:
# Count the sum of missing values for each column
final_merged_df.isna().sum()

In [ ]:
# Count the occurrences of unique values in the 'fraudulent' column
final_merged_df['fraudulent'].value_counts()

*   Text columns are title, location, department, company profile, description, requirements, and benefits, industry, function.
* Categorical columns are employment type, required experience, and required   education
* Numeric variables are salary (low/high), work_remote, has company logo, and has questions
* Column to Predict is fraudulent




In [ ]:
#separate columns based on their type
text_cols = ['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits']
categorical_cols = ['employment_type', 'required_experience', 'required_education', 'industry', 'function']
numeric_cols = ['work_remote', 'has_company_logo', 'has_questions', 'salary_low', 'salary_high']
col_to_predict = ['fraudulent']

*italicized text*### Cleaning Dataset


Text Data

In [ ]:
def move_column(df, col):
  df['Temp_Col'] = df[col]
  df.drop(columns=[col], inplace=True)
  df.rename(columns={'Temp_Col': col}, inplace=True)
  return df

In [ ]:
df['employment_type'].fillna('No Data', inplace=True)
df['required_experience'].fillna('No Data', inplace=True)
df['required_education'].fillna('No Data', inplace=True)
df['industry'].fillna('No Data', inplace=True)
df['function'].fillna('No Data', inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.notna().sum()

In [ ]:
df[df['description'].isna()] #only 1 row with an empty description -- the post only has title and location and it is a fraud post --> keeping it

In [ ]:
#rows that are entirely null in the text columns
null_text = df[(df['location'].isna()) & (df['department'].isna()) & (df['company_profile'].isna()) & (df['requirements'].isna()) & (df['benefits'].isna())] # 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits'
print(f"There are {null_text.shape[0]} rows where all text columns aside from 'title' and 'description' are empty. Of these rows, {null_text['fraudulent'].sum()} are fraudulent posts.")

There are 19 rows where all text columns aside from 'title' and 'description' are empty. Of these rows, 12 are fraudulent posts.

In [ ]:
for col in text_cols: df[col].fillna(value=" ", inplace=True)
print(f"Max length of '{col}': {df[col].map(len).max()}")
    #get maximum number of characters in each column

In [ ]:
#combine all text into 1 column - full_text
df['full_text'] = df['title'] + " " + df['location'] + " " + df['department'] + " " + df['company_profile'] + " " + df['description'] + " " + df['requirements'] + " " + df['benefits']
df['full_text'][0]

In [ ]:

print(f"Min length of 'full text': {df['full_text'].map(len).min()}")
print(f"Max length of 'full text': {df['full_text'].map(len).max()}")

In [ ]:
def update_text(new_df):
  new_df['full_text'] = new_df['full_text'].str.replace(pat='US', repl='USA', case=True)
  new_df['location'] = new_df['location'].str.replace(pat='US', repl='USA', case=True)
  return new_df

In [ ]:
# update US with USA so it does not get mixed up with the word 'us'
df = update_text(df) #replace urls, email, phone numbers (contact details) ?? maybe presence/absence could indicate a fake post? #
df['full_text'] = df['full_text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', ' ', regex=True).replace(r'#PHONE\S+', '', regex=True).replace(r'#EMAIL\S+', '', regex=True)

In [ ]:
#separate 2 separate words that have been put together (i.e. PinterestLoves -> Pinterest Loves)
def space_words(all_text):
  import re
  #print('start', all_text)
  result = re.sub('(?<=[A-Za-z])(?=[A-Z][a-z])', '~', all_text)
  result = re.split('~', result)
  result = ' '.join(result)
  all_text = result # print('end',all_text)
  return all_text

In [ ]:
texts = ['title', 'location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'full_text']
for col in texts:
   df[col] = df[col].apply(space_words)

In [ ]:
df.head(1)

In [ ]:
def get_unique_words(new_df, col):
    unique_words = []
    idx = list(new_df.index.values)
    for i in idx:
        if new_df[col][i]:  # Check if the value is not None
            unique_words.extend(new_df[col][i].split())
    return unique_words

In [ ]:
for col in texts:
  print(f'Number of total words in {col}: {len(get_unique_words(df, col))}')
  print(f'Number of unique words in {col}: {len(set(get_unique_words(df, col)))}\n')

Salary Range Column


In [ ]:
salary_df = df['salary_range'].str.split(pat='-', n=-1, expand=True)
#separate salary range into two columns
salary_df[0].unique()[90:100] #months (strings) are included as salaries due to csv interpretation of the salary range


In [ ]:
#convert the months in the salary columns into integers
months_to_int = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
for i in range(salary_df[0].shape[0]):
  if salary_df[0][i] in months_to_int.keys(): salary_df[0][i] = months_to_int[salary_df[0][i]]
for i in range(salary_df[1].shape[0]):
  if salary_df[1][i] in months_to_int.keys(): salary_df[1][i] = months_to_int[salary_df[1][i]]


In [ ]:
#verify conversion of string months to integers
salary_df[0].unique()[90:100]


In [ ]:
salary_df.fillna(value=-1, inplace=True)
salary_df[0] = np.array(salary_df[0], dtype='int64')
salary_df[1] = np.array(salary_df[1], dtype='int64')
#move the min/max salary columns to the main dataframe
df['salary_low'] = salary_df[0]
df['salary_high'] = salary_df[1]
df.drop(columns= ['salary_range'], inplace=True)
df.drop(columns= ['job_id'], inplace=True)
df['salary_low'].replace(-1, np.nan, inplace=True)
df['salary_high'].replace(-1, np.nan, inplace=True)
df['salary_low'] = df['salary_low'].astype('Int64')
df['salary_high'] = df['salary_high'].astype('Int64')


In [ ]:
df[df['salary_low'].notnull() & df['salary_high'].notnull()][:3] #some examples showing that the salary was converted back to an int value




```
# This is formatted as code
```

Replace Missing Values


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
#split df into train/val/test so I can use the mean of salary_low & salary_high from the training data
print('dataframe shape:',df.shape)
train, test_x = train_test_split(df, train_size=0.8, random_state=1, shuffle=True, stratify=df.fraudulent.values)
print("train shape:",train.shape)
print('test shape:',test_x.shape)
train_x, val_x = train_test_split(train, train_size = 0.8, random_state=1, shuffle = True, stratify=train.fraudulent.values)
print("train_x shape:",train_x.shape)
print('val shape:',val_x.shape)


In [ ]:
train_x['employment_type'].isna().sum()


In [ ]:
df.head()

In [ ]:
train_x.groupby('employment_type').mean()


In [ ]:
salary_means = {}
employment_types = list(train_x['employment_type'].unique())
employment_types.sort()
employment_types


In [ ]:
low_means = list(train_x.groupby('employment_type')['salary_low'].mean())
high_means = list(train_x.groupby('employment_type')['salary_high'].mean())
salary_means = {}
for i in range(len(employment_types)):
  salary_means[employment_types[i]] = [int(low_means[i]), int(high_means[i])]
print("List of Means (from train_x dataset):", salary_means)


In [ ]:
for et in employment_types: temp = df[df['employment_type'] == et] #separate the group temp['salary_low'].fillna(salary_means[et][0], inplace=True) temp['salary_high'].fillna(salary_means[et][1], inplace=True) df.update(temp, overwrite=False) df['salary_low'] = df['salary_low'].astype('int64') df['salary_high'] = df['salary_high'].astype('int64') print(df.shape) df.isna().sum()


In [ ]:
df.info() #confirms that the salary columns are int64 and not float64


Additional Text Processing


In [ ]:
df_preproc_done = df.copy()
df_preproc_done.columns


In [ ]:
# combine all text columns into the title column # drop all text columns except full_text # rename 'title' column to 'full_text'
df['title'] = df['full_text']
df.drop(columns=['location', 'department', 'company_profile', 'description', 'requirements', 'benefits', 'full_text'], inplace=True)
df.rename(columns={'title': 'full_text'}, inplace=True) #put fraud column at the end of the dataframe
df = move_column(df, 'fraudulent')
df.head(1)


Completed:


*   replaced NA values in categorical columns with 'No Data'
*   created 'full_text' column with all textual columns combined
*   split the 'salary_range' column into 2 for the range --> salary_low, salary_high
*   replace 'months' in salary columns with integers
*   replace NA values in salary columns with -1, create new columns in main dataframe for salary (low/high), replace -1 with NaN and cast to type Integer64
*   move full_text to the front of the dataframe, move fraudulent to the end of the dataframe; and rename both columns






